- cleanup for some bag of words issues (unicode cruft)
- extract the namespaces (should be a better way than this, but *waves*)


oh. well. we simply can't handle the emoji: http://www.unicode.org/L2/L2015/15049-C-emoji-annotations.xml


In [24]:
import os
import glob
import json
from lxml import etree

#files = glob.glob('../solr_superset/docs/*.json')

# let's strip out all of the namespaces and make a stopwords list for the identifiers

files = glob.glob('../solr_superset/identified/*.json')

master_namespaces = set()

for f in files:
    with open(f, 'r') as g:
        data = json.loads(g.read())
    
    content = data['content'].encode('unicode_escape')
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
    
    
    document_namespaces = dict(xml.xpath('/*/namespace::*'))
    if None in document_namespaces:
        document_namespaces['default'] = document_namespaces[None]
        del document_namespaces[None]

    # now run through any child namespace issues
    all_namespaces = xml.xpath('//namespace::*')
    for i, ns in enumerate(all_namespaces):
        if ns[1] in document_namespaces.values():
            continue
        new_key = ns[0] if ns[0] else 'default%s' % i
        document_namespaces[new_key] = ns[1]

    # and get the values
    found_namespaces = set(document_namespaces.values())
        
    master_namespaces = master_namespaces.symmetric_difference(found_namespaces)

#master_namespaces
with open('../solr_superset/namespaces.txt', 'w') as f:
    f.write('\n'.join(list(master_namespaces)))

In [22]:
import re

pttn = ur'[\\{2,}ufffd]'

some_text = '''\\\\ufffd\\ufffd\\\\ufffds</annotation> dd  <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd{#\\ufffd\\ufffd\\ufffd}{0\\ufffd\\ufffd\\ufffd}{1\\ufffd\\ufffd\\ufffd}{2\\ufffd\\ufffd\\ufffd}{3\\ufffd\\ufffd\\ufffd}{4\\ufffd\\ufffd\\ufffd}{5\\ufffd\\ufffd\\ufffd}{6\\ufffd\\ufffd\\ufffd}{7\\ufffd\\ufffd\\ufffd}{8\\ufffd\\ufffd\\ufffd}{9\\ufffd\\ufffd\\ufffd}]'>word</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd]'>worker</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd]'>world</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd]'>worried</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd]'>wrench</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd]'>write</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd]'>yellow</annotation>   <annotation cp='[{\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd}]'>yemen</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd]'>yen</annotation>   <annotation cp='[{\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd}]'>zambia</annotation>   <annotation cp='[{\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd}]'>zimbabwe</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd-\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd]'>zodiac</annotation>   <annotation cp='[\\ufffd\\ufffd\\ufffd\\ufffd]'>zzz</annotation>  </annotations> </ldml>'''

#new_text = re.sub(pttn, ' ', some_text)
new_text = some_text.replace('\\\\ufffd', ' ').replace('\\ufffd', ' ')

' '.join(new_text.split())

#some_text.replace(u'\\\\ufffd', ' ').replace(u'\\ufffd', ' ')


"s</annotation> dd <annotation cp='[ - - - - {# }{0 }{1 }{2 }{3 }{4 }{5 }{6 }{7 }{8 }{9 }]'>word</annotation> <annotation cp='[ ]'>worker</annotation> <annotation cp='[ - ]'>world</annotation> <annotation cp='[ ]'>worried</annotation> <annotation cp='[ ]'>wrench</annotation> <annotation cp='[ ]'>write</annotation> <annotation cp='[ ]'>yellow</annotation> <annotation cp='[{ }]'>yemen</annotation> <annotation cp='[ ]'>yen</annotation> <annotation cp='[{ }]'>zambia</annotation> <annotation cp='[{ }]'>zimbabwe</annotation> <annotation cp='[ - ]'>zodiac</annotation> <annotation cp='[ ]'>zzz</annotation> </annotations> </ldml>"

In [6]:
some_text = '''the volume distribution $\\rm(\\langle V\\rangle,\\langle V\\sp3\\rangle,\\...).$ As long as the ensemble'''

pttn = ur'[\\{2,}]'

new_text = re.sub(pttn, ' ', some_text)

' '.join(new_text.split())



'the volume distribution $ rm( langle V rangle langle V sp3 rangle ...).$ As long as the ensemble'

In [8]:
text = 'here\'s some, stuff [hi] but : no'

simple_pattern = r'[;|>+:=.,()/?!\[\]]'

re.sub(simple_pattern, ' ', text)

"here's some  stuff  hi  but   no"

In [25]:
# let's play with numbers

some_text = '''some cat's 435634 are things. -34252'''

pttn = ur'\w*\b-?\d\s*\w*'

captures = re.findall(pttn, some_text)

stuff = re.sub('|'.join(captures), ' ', some_text)

pttn = "cat's"

re.sub(pttn, '***', stuff)



'some ***   are things. - '

In [27]:
with open('../lib/corpus/mimetypes.txt', 'r') as f:
    mimes = [a.strip() for a in f.readlines()]
    
for m in mimes:
    try:
        x = re.compile(m)
    except Exception as ex:
        print m
        print '\t', ex

In [55]:
# let's play with the bag of words steps because that is just annoying
# also bad tests. meep.

import re
import collections
import json
import HTMLParser
import nltk
import yaml
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import WordListCorpusReader
from nltk.stem.wordnet import WordNetLemmatizer
from itertools import chain


_corpus_root = '../lib/corpus'


def flatten(items, excluded_keys=[]):
    def _flatten(item):
        if isinstance(item, dict):
            for k, v in item.iteritems():
                if k in excluded_keys:
                    continue
                # TODO: this introduces nested lists again!
                yield list(_flatten(v))
        elif isinstance(item, list):
            for i in item:
                if isinstance(i, collections.Iterable) and not isinstance(i, basestring):
                    for subitem in _flatten(i):
                        yield subitem
                else:
                    yield i
        elif isinstance(item, str):
            yield item

    arr = list(_flatten(items))
    if len([isinstance(a, collections.Iterable) and
            not isinstance(a, basestring) for a in arr]) > 0:
        # ick, flatten the dict issue again.
        return list(_flatten(arr))

    return arr

def remove_punctuation(text):
    simple_pattern = r'[;|>+:=.,()/?!\[\]{}]'
    return re.sub(simple_pattern, ' ', text)


def split_words(text):
    ''' '''
    simple_pattern = r'[/:.]'
    return re.split(simple_pattern, text)


def remove_stopwords(text):
    _stopwords = set(stopwords.words('english'))
    words = word_tokenize(text)
    words = words if isinstance(words, list) else words.split()
    return ' '.join([w for w in words if w not in _stopwords and w])


def remove_mimetypes(text):
    mimetypes = WordListCorpusReader(_corpus_root, 'mimetypes.txt')
    words = [w.replace('+', '\+') for w in mimetypes.words()]

    pttn = re.compile('|'.join(words))
    return pttn.sub(' ', text)


def remove_numeric(text):
    match_pttn = ur'\w*\b-?\d\s*\w*'
    captures = re.findall(match_pttn, text)

    # strip them out
    if captures:
        return re.sub(re.compile('|'.join(captures)), ' ', text)

    return text


def extract_mimetypes(text, do_replace=True):
    mimetypes = WordListCorpusReader(_corpus_root, 'mimetypes.txt')

    found_mimetypes = [w for w in mimetypes.words() if w in text]

    if do_replace:
        text = remove_mimetypes(text)

    return found_mimetypes, text


def collapse_to_bag(data_blob, exclude_urls=True):
    _url_keys = ['url']
    excludes = _url_keys if exclude_urls else []

    # TODO: run the generator
    bag_of_words = flatten(data_blob, excludes)
    bag_of_words = bag_of_words if isinstance(bag_of_words, list) else bag_of_words.split()

    return ' '.join(bag_of_words)


def normalize_subjects(subjects, do_split=False, return_unique=True):
    if not subjects:
        return

    normalized_subjects = []
    for subject in subjects:
        if not subject:
            # skip any empty tag
            continue
        normalized = normalize_keyword_text(subject)
        normalized_subjects += [n.strip() for n in normalized.split(',') if n] \
            if do_split else [normalized]

    return list(set(normalized_subjects)) if return_unique else normalized_subjects


def normalize_keyword_text(keyword_string):
    if not keyword_string:
        return

    # unescape
    hp = HTMLParser.HTMLParser()
    keyword_string = hp.unescape(keyword_string)

    # replace underscores (assume these are NOT delimiters
    #    but phrase concatenators)
    underscore_pattern = r'[_]'
    keyword_string = re.sub(underscore_pattern, ' ', keyword_string)

    punctuation_pattern = r'[;|>+:=+(){}]'
    return re.sub(punctuation_pattern, ',', keyword_string)

def _normalize_keywords(service_description):
    service = service_description.get('service', {})
    if not service:
        return service_description
    subjects = service.get('subject', [])
    if not subjects:
        return service_description

    # return split and as a unique list
    service['subject'] = normalize_subjects(subjects, True, True)
    service_description['service'] = service
    return service_description

def parse_yaml(yaml_file):
    with open(yaml_file, 'r') as f:
        y = yaml.load(f.read())
    return y

def extract_task_config(yaml_config, task):
    return yaml_config.get(task, {})


In [56]:
# and the actual running of the thing

config = parse_yaml('../tasks/bagofwords_from_parsed_20150504.yaml')
config = extract_task_config(config, 'BagOfWordsFromParsed')
tasks = config.get('tasks', {})

print tasks
print 

with open('../solr_superset/parsed/006bc0af7c797ea4636812a8e3154178_parsed.json', 'r') as f:
    data = json.loads(f.read())
    
service_description = data.get('service_description', {})

service_description = _normalize_keywords(service_description)

#service_description['service']['subject']
    
# it's not here    
bow = collapse_to_bag(data)

for k, v in tasks.iteritems():
    if k == 'remove_mimetypes':
        bow = remove_mimetypes(bow)
    elif k == 'remove_punctuation':
        bow = remove_punctuation(bow)
    elif k == 'remove_stopwords':
        bow = remove_stopwords(bow)
    elif k == 'remove_numeric':
        bow = remove_numeric(bow)
    
    print k
    print '\t', bow
    print



{'normalize_keywords': True, 'remove_mimetypes': True, 'remove_stopwords': True, 'remove_numeric': True, 'remove_punctuation': True}

normalize_keywords
	Pacific Science Center inspires a lifelong interest in science, math and technology by engaging diverse communities through interactive and innovative exhibits and programs. Pacific Science Center

remove_mimetypes
	Pacific Science Center inspires a lifelong interest in science, math and technology by engaging diverse communities through interactive and innovative exhibits and programs. Pacific Science Center

remove_stopwords
	Pacific Science Center inspires lifelong interest science , math technology engaging diverse communities interactive innovative exhibits programs . Pacific Science Center

remove_numeric
	Pacific Science Center inspires lifelong interest science , math technology engaging diverse communities interactive innovative exhibits programs . Pacific Science Center

remove_punctuation
	Pacific Science Center inspires li

In [54]:
x = '''Pacific Science Center inspires a lifelong interest in science, math and technology by 
engaging diverse communities through interactive and innovative exhibits and programs. Pacific Science Center'''

match_pttn = ur'\w*\b-?\d\s*\w*'
captures = re.findall(match_pttn, text)

print captures

if captures:
    re.sub(re.compile('|'.join(captures)), ' ', x)

[]
